# Задача

Нужно найти, какой из репетиторов получал самую низкую среднюю оценку качества после уроков по физике за каждый день, и вывести эту оценку.

In [1]:
import pandas as pd

In [2]:
lessons = pd.read_csv('lessons.txt', engine='python', sep = '\s*[|]\s*', parse_dates = ['scheduled_time'], skiprows = [1],
                     skipfooter = 2)

In [3]:
lessons.head()

,id,event_id,subject,scheduled_time
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 12:00:00
1,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 13:00:00
2,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 13:00:00
3,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 12:00:00
4,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 14:00:00


In [4]:
participants = pd.read_csv('participants.txt', engine='python', sep = '\s*[|]\s*', skiprows = [1], skipfooter = 2)

In [5]:
participants.head()

,event_id,user_id
0,38114,e28351f5-4ccb-4549-8647-d43f2b15e7b8
1,38114,4df2832a-1d63-4453-9659-43993fc35996
2,51568,bb1c0bc8-1212-452b-97a0-439d4a2169e2
3,51568,63441abe-c4da-4275-ba26-66f7dbd65dde
4,52790,a1323b68-c82c-429a-8e2c-65597e648c1a


In [6]:
quality = pd.read_csv('quality.txt', engine='python', sep = '\s*[|]\s*', skiprows = [1], skipfooter = 2)

In [7]:
quality.head()

,lesson_id,tech_quality
0,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,5.0
1,62e1a078-33de-47c1-99d2-845b1daca56f,5.0
2,62e1a078-33de-47c1-99d2-845b1daca56f,5.0
3,00fc6685-f53a-49bb-b960-5e0042fd3852,5.0
4,00fc6685-f53a-49bb-b960-5e0042fd3852,5.0


In [8]:
users = pd.read_csv('users.txt', engine='python', sep = '\s*[|]\s*', skiprows = [1], skipfooter = 2)

In [9]:
users.head()

,id,role
0,e28351f5-4ccb-4549-8647-d43f2b15e7b8,pupil
1,4df2832a-1d63-4453-9659-43993fc35996,tutor
2,bb1c0bc8-1212-452b-97a0-439d4a2169e2,pupil
3,63441abe-c4da-4275-ba26-66f7dbd65dde,tutor
4,a1323b68-c82c-429a-8e2c-65597e648c1a,tutor


In [10]:
tutor_eventid = users[users.role == 'tutor'].merge(participants, left_on='id', right_on='user_id',
                                              how='inner')[['user_id', 'event_id']]
tutor_eventid.drop_duplicates(inplace=True)

In [11]:
tutor_eventid.head()

,user_id,event_id
0,4df2832a-1d63-4453-9659-43993fc35996,38114
1,4df2832a-1d63-4453-9659-43993fc35996,38111
2,4df2832a-1d63-4453-9659-43993fc35996,56195
3,4df2832a-1d63-4453-9659-43993fc35996,50880
5,4df2832a-1d63-4453-9659-43993fc35996,51450


In [12]:
qual_less = quality.merge(lessons, left_on='lesson_id', right_on='id',
                          how='inner')[['tech_quality', 'event_id', 'scheduled_time', 'subject']]
qual_less = qual_less[qual_less.subject == 'phys'][['tech_quality', 'event_id', 'scheduled_time']]

In [13]:
qual_less.head()

,tech_quality,event_id,scheduled_time
3,5.0,51341,2020-01-17 12:00:00
4,5.0,51341,2020-01-17 12:00:00
5,5.0,55048,2020-01-19 14:00:00
6,5.0,53026,2020-01-19 15:00:00
8,5.0,51645,2020-01-19 14:00:00


In [14]:
qual_time_tutor = qual_less.merge(tutor_eventid, left_on='event_id', right_on='event_id',
                                   how='inner')[['tech_quality', 'scheduled_time', 'user_id']]
qual_time_tutor.dropna(inplace=True)
qual_time_tutor.sort_values(by='scheduled_time', inplace=True)

In [15]:
qual_time_tutor

,tech_quality,scheduled_time,user_id
41,5.0,2020-01-11 09:00:00,c0c073c8-cb9c-4ebc-ac13-472c06029591
32,5.0,2020-01-11 09:00:00,30a19496-bdaf-461c-abbc-2709ae520201
39,5.0,2020-01-11 10:00:00,43efce48-94b2-4412-857f-223d45969008
18,5.0,2020-01-11 11:00:00,43efce48-94b2-4412-857f-223d45969008
17,5.0,2020-01-11 11:00:00,43efce48-94b2-4412-857f-223d45969008
...,...,...,...
31,5.0,2020-01-20 15:00:00,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6
53,5.0,2020-01-20 16:45:00,be676776-8366-4c71-8a35-d58014806eb5
52,5.0,2020-01-20 16:45:00,be676776-8366-4c71-8a35-d58014806eb5
6,5.0,2020-01-20 17:00:00,696c838e-c054-4e9f-a51a-50bf5660f364


In [16]:
qual_time_tutor['scheduled_time'] += pd.Timedelta('3 hours')
qual_time_tutor['scheduled_time'] = qual_time_tutor['scheduled_time'].map(lambda x: x.date())

In [17]:
qual_time_tutor

,tech_quality,scheduled_time,user_id
41,5.0,2020-01-11,c0c073c8-cb9c-4ebc-ac13-472c06029591
32,5.0,2020-01-11,30a19496-bdaf-461c-abbc-2709ae520201
39,5.0,2020-01-11,43efce48-94b2-4412-857f-223d45969008
18,5.0,2020-01-11,43efce48-94b2-4412-857f-223d45969008
17,5.0,2020-01-11,43efce48-94b2-4412-857f-223d45969008
...,...,...,...
31,5.0,2020-01-20,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6
53,5.0,2020-01-20,be676776-8366-4c71-8a35-d58014806eb5
52,5.0,2020-01-20,be676776-8366-4c71-8a35-d58014806eb5
6,5.0,2020-01-20,696c838e-c054-4e9f-a51a-50bf5660f364


In [18]:
answer = qual_time_tutor.groupby(['scheduled_time', 'user_id']).mean().sort_values('tech_quality')

In [19]:
answer.head(5)

,,tech_quality
scheduled_time,user_id,
2020-01-16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.000000
2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.000000
2020-01-18,43efce48-94b2-4412-857f-223d45969008,4.250000
2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.428571
2020-01-20,43efce48-94b2-4412-857f-223d45969008,4.500000


In [20]:
answer.index[0][1], answer.tech_quality[0]

('2fa2ab62-f1b0-4036-872f-bcfd9a8686ff', 4.0)